# RECONOCIMIENTO DE EXPRESIONES FACIALES 

Librerias 

In [52]:
import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import random

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix

In [37]:
# Cargar los archivos CSV
train_df = pd.read_csv('./data/train_set.csv')
test_df = pd.read_csv('./data/test_set.csv')

In [38]:
# Función para cargar y preprocesar imágenes
def load_and_preprocess_image(image_path, target_size=(48, 48)):
    try:
        image = load_img(image_path, color_mode='grayscale', target_size=target_size)
        image = img_to_array(image)
        image = image / 255.0  # Normalizar los valores de los píxeles
        return image
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

In [39]:
# Función para cargar imágenes y etiquetas desde una carpeta
def load_images_from_folder(folder_path):
    images = []
    labels = []
    label_map = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}  # Asignar etiquetas numéricas a las emociones
    for emotion in os.listdir(folder_path):
        emotion_folder = os.path.join(folder_path, emotion)
        if os.path.isdir(emotion_folder):
            for img_name in os.listdir(emotion_folder):
                img_path = os.path.join(emotion_folder, img_name)
                if os.access(img_path, os.R_OK):  # Verificar permisos de lectura
                    image = load_and_preprocess_image(img_path)
                    if image is not None:
                        images.append(image)
                        labels.append(label_map[emotion])
                else:
                    print(f"Permission denied for image {img_path}")
    return np.array(images), np.array(labels)

In [40]:
# Cargar las imágenes de entrenamiento y las etiquetas
train_images, train_labels = load_images_from_folder('./data/images/train')

In [41]:
# Convertir las etiquetas a formato categórico
train_labels = to_categorical(train_labels, num_classes=7)

In [42]:
# Dividir en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [43]:
print(f"Train images shape: {X_train.shape}, Train labels shape: {y_train.shape}")
print(f"Validation images shape: {X_val.shape}, Validation labels shape: {y_val.shape}")

Train images shape: (23056, 48, 48, 1), Train labels shape: (23056, 7)
Validation images shape: (5765, 48, 48, 1), Validation labels shape: (5765, 7)


In [44]:
# Cargar las imágenes de prueba
def load_test_images(folder_path):
    images = []
    image_names = []
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        if os.access(img_path, os.R_OK):  # Verificar permisos de lectura
            image = load_and_preprocess_image(img_path)
            if image is not None:
                images.append(image)
                image_names.append(img_name)
        else:
            print(f"Permission denied for image {img_path}")
    return np.array(images), image_names



In [45]:
test_images, test_image_names = load_test_images('./data/images/test')
print(f"Test images shape: {test_images.shape}")

Error loading image ./data/images/test\temp: [Errno 13] Permission denied: './data/images/test\\temp'
Test images shape: (7066, 48, 48, 1)


In [46]:
def create_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax')  # Usamos softmax para la clasificación de 7 clases
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [47]:
# Definir el tamaño de entrada basado en nuestras imágenes
input_shape = (48, 48, 1)
model = create_cnn_model(input_shape)
model.summary()

c:\Users\admin\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 355,847 (1.36 MB)

 Trainable params: 355,847 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

In [48]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.2317 - loss: 1.8310 - val_accuracy: 0.3547 - val_loss: 1.6395
Epoch 2/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.3518 - loss: 1.6481 - val_accuracy: 0.4295 - val_loss: 1.4734
Epoch 3/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.4113 - loss: 1.5227 - val_accuracy: 0.4852 - val_loss: 1.3747
Epoch 4/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.4397 - loss: 1.4392 - val_accuracy: 0.4996 - val_loss: 1.3214
Epoch 5/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.4679 - loss: 1.3735 - val_accuracy: 0.5103 - val_loss: 1.2835
Epoch 6/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.4840 - loss: 1.3453 - val_accuracy: 0.5176 - val_loss: 1.2619
Epoch 7/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.4988 - loss: 1.3175 - val_accuracy: 0.5277 - val_loss: 1.2282
Epoch 8/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.5077 - loss: 1.2928 - 

In [49]:
# Evaluar el modelo en el conjunto de validación
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

181/181 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5798 - loss: 1.1234
Validation Loss: 1.1344196796417236, Validation Accuracy: 0.575888991355896


In [53]:
# Realizar predicciones en el conjunto de pruebas
test_predictions = model.predict(test_images)
test_predictions = np.argmax(test_predictions, axis=1)  # Convertir a etiquetas

221/221 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [59]:
# Crear un diccionario inverso para convertir etiquetas numéricas a etiquetas de emociones
inv_label_map = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

In [60]:
# Convertir etiquetas numéricas a etiquetas de emociones
test_predictions_labels = [inv_label_map[pred] for pred in test_predictions]

In [61]:
# Crear el archivo de submission
submission_df = pd.DataFrame({'image': test_image_names, 'emotion': test_predictions_labels})
submission_df.to_csv('submission.csv', index=False)
print(submission_df.head())


       image   emotion
0  10004.jpg     angry
1  10019.jpg     happy
2  10023.jpg       sad
3  10029.jpg  surprise
4   1003.jpg       sad
